In [ ]:
import sympy as sp
from sympy import *
import mpmath
import multiprocessing
import math
import numpy as np
import pandas as pd
from functools import reduce
from sympy import cos, sin, sqrt, exp, pi

# Function to calculate the least common multiple of two numbers
def lcm(a, b):
    return abs(a * b) // sp.gcd(a, b)

# Function to calculate the LCM of multiple numbers
def lcm_multiple(numbers):
    return reduce(lcm, numbers)

accuracy = 60

# Initialize variables
sigma1 = sp.Rational('75')
sigma2 = sp.Rational('5')
sigma3 = sp.Rational('15')
sigma4 = sp.Rational('10')
sigma5 = sp.Rational('6')
sigma6 = sp.Rational('450')

# Convert sigmas to integers for LCM calculation
sigma1_int = int(sigma1)
sigma2_int = int(sigma2)
sigma3_int = int(sigma3)
sigma4_int = int(sigma4)
sigma5_int = int(sigma5)
sigma6_int = int(sigma6)

# Calculate rho values for each sigma
rho_1 = sp.N(1/(2 * sigma1) * 9, accuracy)
rho_2 = sp.N(1/(2 * sigma2) * 9, accuracy)
rho_3 = sp.N(1/(2 * sigma3) * 9, accuracy)
rho_4 = sp.N(1/(2 * sigma4) * 9 * 3, accuracy)
rho_5 = sp.N(1/(2 * sigma5) * 9, accuracy)
rho_6 = sp.N(1/(2 * sigma6) * 9, accuracy)

# Calculate total rho
rho_total = sp.N(rho_1 + rho_2 + rho_3 + rho_4 + rho_5 + rho_6, accuracy)

# Specify delta
delta = sp.Rational('1E-10')

# Calculating epsilon
eps = sp.N(rho_total + 2 * sqrt(-rho_total * sp.log(delta)), accuracy)

# Perform calculations
L = lcm_multiple([sigma1_int, sigma2_int, sigma3_int, sigma4_int, sigma5_int, sigma6_int])

teps_first = sp.N(eps - 4.5 * (1/sigma1 + 1/sigma2 + 1/sigma3 + 3/sigma4 + 1/sigma5 + 1/sigma6), accuracy)
teps_second = sp.N(eps + 4.5 * (1/sigma1 + 1/sigma2 + 1/sigma3 + 3/sigma4 + 1/sigma5 + 1/sigma6), accuracy)

N = sp.ceiling(teps_first * L)
N_2 = sp.ceiling(teps_second * L)

def char_func(sigma, hatsigma, t):
    u_range = range(1, int(sp.ceiling(20 * sqrt(9 * sigma))) + 1)
    sum_exp_cos = sum(sp.N(exp(-u**2 / (2 * 9 * sigma)) * cos(hatsigma * t * u), accuracy) for u in u_range)
    factor = sp.N(1 / sqrt(2 * pi * 9 * sigma), accuracy)
    return (1 + 2 * sum_exp_cos) * factor

def char_func_4(t):
    u_range = range(1, int(sp.ceiling(20 * sqrt(9 * 3 * sigma4))) + 1)
    sum_exp_cos = sum(sp.N(exp(-u**2 / (2 * 9 * 3 * sigma4)) * cos(L/sigma4 * t * u), accuracy) for u in u_range)
    factor = sp.N(1 / sqrt(2 * pi * 9 * 3 * sigma4), accuracy)
    return (1 + 2 * sum_exp_cos) * factor

# Precomputed weighted variances
sigmas = [sigma1, sigma2, sigma3, sigma5, sigma6]

# Precomputed weighted variances
hatsigmas = [L / sigma for sigma in [sigma1, sigma2, sigma3, sigma5, sigma6]]

# Define CHAR
def CHAR(t):
    product = 1
    for sigma in sigmas:
        product *= char_func(sigma, L / sigma, t)
    product *= char_func_4(t)
    return product

# Define weight functions
def weight_first(t):
    if t != 0:
        return (cos((3 * N_2 + 1) * t/2) * sin(3 * N_2 * t/2) / sin(t/2) - cos(N * t/2) * sin((N-1) * t/2) / sin(t/2))
    else:
        return 3 * N_2 - N + 1

def weight_second(t):
    if t != 0:
        return (cos((3 * N_2 + 1) * t/2) * sin(3 * N_2 * t/2) / sin(t/2) - cos(N_2 * t/2) * sin((N_2-1) * t/2) / sin(t/2))
    else:
        return 2 * N_2 + 1

# Define the expression to be integrated
def tobeint_first(t):
    return sp.N(1/(pi) * CHAR(t) * weight_first(t), accuracy)

# Define the expression to be integrated
def tobeint_second(t):
    return sp.N(1/(pi) * CHAR(t) * weight_second(t), accuracy)

# Set the range and intervals for your calculations
start = 0
end = float(sp.pi)  # Convert Sympy pi to a float
num_intervals = 451
delta_x = 1 / (num_intervals - 1)

# Initialize an empty list to store the results
results = []

# Iterate and store results instead of printing
for x in np.linspace(start, end, int((end - start) / delta_x) + 1)[:-1]:
    x_sp = sp.N(x, accuracy)

    # Print the values of char_func and char_func_4 with high precision
    print(sp.N(char_func(sigmas[0], hatsigmas[0], x_sp), accuracy))
    # print(sp.N(char_func(sigmas[1], hatsigmas[1], x_sp), accuracy))
    # print(sp.N(char_func(sigmas[2], hatsigmas[2], x_sp), accuracy))
    # print(sp.N(char_func_4(x_sp), accuracy))
    # print(sp.N(char_func(sigmas[3], hatsigmas[3], x_sp), accuracy))
    print(sp.N(char_func(sigmas[4], hatsigmas[4], x_sp), accuracy))
    print()

    temp_results = [
        x_sp,
        sp.N(char_func(sigmas[0], hatsigmas[0], x_sp), accuracy),
        sp.N(char_func(sigmas[1], hatsigmas[1], x_sp), accuracy),
        sp.N(char_func(sigmas[2], hatsigmas[2], x_sp), accuracy),
        sp.N(char_func_4(x_sp), accuracy),
        sp.N(char_func(sigmas[3], hatsigmas[3], x_sp), accuracy),
        sp.N(char_func(sigmas[4], hatsigmas[4], x_sp), accuracy)
    ]
    results.append(temp_results)

# Convert the results list to a DataFrame
df = pd.DataFrame(results, columns=['x', 'Char_Func_1', 'Char_Func_2', 'Char_Func_3', 'Char_Func_4', 'Char_Func_5', 'Char_Func_6'])
print(df)

# Save the DataFrame as a CSV file in your Google Drive
file_path = '/content/drive/MyDrive/Research/Discrete Gaussian Mechanism/Characteristic.csv'  # Change to your specific folder
df.to_csv(file_path, index=False)


0.999999999999999999999999999999999999999999999999999999999999
1.00000000000000000000000000000000000000000000000000000000000

0.941707199595080658502958306237911181498915983834416515978490
0.990039787900062771764191603589611599803790207480649672980722

0.786436321065665980396322138280355521072379357362096981708964
0.960750443948264017427273534558714997810293069960410240043979

0.582429034234468256787691319609708297736100638315185524909395
0.913847727270914121178837191768042348586949365686501366475227

0.382520092052809489024571753593829337304138728249081693904257
0.852005454814615209181375216540068431044129574191807733243612

0.222790807789597685087445301886615389468692238920271075723894
0.778603248504757599996470876176568190969890899887266345906898

0.115072637173262591074286571417149911442389111979439271922640
0.697421520471727530797734964848433238581225324783376112845732

0.0527082558749514420052142702799907784370237793206180614461522
0.6123218743238580922850335653241666072797202679